# Final Project: Disaster Tweet Classification

In [314]:
__author__ = "Kevin Guo, Pranav Sriram, Raymond Yao"
__version__ = "CS224u, Stanford, Spring 2021"

## Data Pre-Processing

In [68]:
import numpy as np
import pandas as pd
import re
from transformers import BertModel, BertTokenizer, DistilBertTokenizer, DistilBertForSequenceClassification
import utils
import torch
import torch.nn as nn
from torch_shallow_neural_classifier import TorchShallowNeuralClassifier
from datasets import Dataset
from datasets import load_dataset
from transformers import TrainingArguments
from transformers import Trainer
from torch.utils.data import DataLoader
from transformers import AdamW
from transformers import get_scheduler
from tqdm.auto import tqdm
from torch.autograd import Variable

In [3]:
# Set random seeds
utils.fix_random_seeds()

In [4]:
df = pd.read_csv('tweets_mod_copy.csv')
df.head(10)

,text,labels
0,"Communal violence in Bhainsa, Telangana. ""Ston...",1
1,Telangana: Section 144 has been imposed in Bha...,1
2,Arsonist sets cars ablaze at dealership https:...,1
3,Arsonist sets cars ablaze at dealership https:...,1
4,"""Lord Jesus, your love brings freedom and pard...",0
5,"If this child was Chinese, this tweet would ha...",0
6,Several houses have been set ablaze in Ngemsib...,1
7,Asansol: A BJP office in Salanpur village was ...,1
8,"National Security Minister, Kan Dapaah's side ...",0
9,This creature who’s soul is no longer clarent ...,0


In [5]:
# Remove URLs
df['text'] = df['text'].apply(lambda x: re.sub(r'https?\S+', '', x))
df.head(10)

,text,labels
0,"Communal violence in Bhainsa, Telangana. ""Ston...",1
1,Telangana: Section 144 has been imposed in Bha...,1
2,Arsonist sets cars ablaze at dealership,1
3,Arsonist sets cars ablaze at dealership,1
4,"""Lord Jesus, your love brings freedom and pard...",0
5,"If this child was Chinese, this tweet would ha...",0
6,Several houses have been set ablaze in Ngemsib...,1
7,Asansol: A BJP office in Salanpur village was ...,1
8,"National Security Minister, Kan Dapaah's side ...",0
9,This creature who’s soul is no longer clarent ...,0


In [6]:
# Split into train, dev, and test sets using a 80-10-10 ratio
# train = df 
# dev = df
# test = df
train, dev, test = np.split(df.sample(frac=1, random_state=42), [int(.8*len(df)), int(.9*len(df))])

In [7]:
# train_dataset = load_dataset('csv', data_files = 'tweets.csv')
# dev_dataset = load_dataset('csv', data_files = 'tweets.csv')
# test_dataset = load_dataset('csv', data_files = 'tweets.csv')

train_dataset = Dataset.from_pandas(train)
dev_dataset = Dataset.from_pandas(dev)
test_dataset = Dataset.from_pandas(test)
# dataset = Dataset.from_pandas(df)

In [69]:
# from transformers import AutoModelForSequenceClassification
# # weights_name = 'bert-base-cased'
# # tokenizer = BertTokenizer.from_pretrained(weights_name)
# from transformers import AutoTokenizer

# tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
# model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)
# # x_train = train_dataset['text']
# # y_train = train_dataset['target']

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

In [74]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_dev_dataset = dev_dataset.map(tokenize_function, batched=True)
tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True)
# tokenized_train_dataset = tokenized_train_dataset.remove_columns(["text"])
# tokenized_train_dataset = tokenized_train_dataset.remove_columns(["__index_level_0__"])
# tokenized_train_dataset = tokenized_train_dataset.remove_columns(["labels"])


tokenized_train_dataset.set_format("torch")

tokenized_dev_dataset.set_format("torch")

tokenized_test_dataset.set_format("torch")
tokenized_train_dataset

Dataset({
    features: ['__index_level_0__', 'attention_mask', 'input_ids', 'labels', 'text'],
    num_rows: 9096
})

In [75]:
training_args = TrainingArguments("test_trainer")
trainer = Trainer(
    model=model, args=training_args, train_dataset=tokenized_train_dataset, eval_dataset=tokenized_dev_dataset
)

In [76]:
train_dataloader = DataLoader(tokenized_train_dataset, shuffle=True, batch_size=8)
eval_dataloader = DataLoader(tokenized_dev_dataset, batch_size=8)

In [77]:
trainer.train()

Step,Training Loss


KeyboardInterrupt: 

In [62]:
optimizer = AdamW(model.parameters(), lr=5e-5)
num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

In [63]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0): TransformerBlock(
        (attention): MultiHeadSelfAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): Linear(i

In [65]:
progress_bar = tqdm(range(num_training_steps))

bert_model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = bert_model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

AttributeError: 'BaseModelOutputWithPoolingAndCrossAttentions' object has no attribute 'loss'